In [ ]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
 
 
class CpEvent:
    def set_params(self, client):
        self.client = client
 
    def OnReceived(self):
        code = self.client.GetHeaderValue(0)  # 종목코도
        name = self.client.GetHeaderValue(1)  # 종목명
        timess = self.client.GetHeaderValue(18)  # 초
        exFlag = self.client.GetHeaderValue(19)  # 예상체결 플래그
        cprice = self.client.GetHeaderValue(13)  # 현재가
        diff = self.client.GetHeaderValue(2)  # 대비
        cVol = self.client.GetHeaderValue(17)  # 순간체결수량
        vol = self.client.GetHeaderValue(9)  # 거래량
 
        if (exFlag == ord('1')):  # 동시호가 시간 (예상체결)
            print("실시간(예상체결)", name, timess, "*", cprice, "대비", diff, "체결량", cVol, "거래량", vol)
        elif (exFlag == ord('2')):  # 장중(체결)
            print("실시간(장중 체결)", name, timess, cprice, "대비", diff, "체결량", cVol, "거래량", vol)
 
 
class CpStockCur:
    def Subscribe(self, code):
        self.objStockCur = win32com.client.Dispatch("DsCbo1.StockCur")
        handler = win32com.client.WithEvents(self.objStockCur, CpEvent)
        self.objStockCur.SetInputValue(0, code)
        handler.set_params(self.objStockCur)
        self.objStockCur.Subscribe()
 
    def Unsubscribe(self):
        self.objStockCur.Unsubscribe()
 
 
class MyWindow(QMainWindow):
 
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PLUS API TEST")
        self.setGeometry(300, 300, 300, 150)
        self.isSB = False
        self.objStockCur1 = CpStockCur()
        self.objStockCur2 = CpStockCur()
        self.objStockCur3 = CpStockCur()
 
        btnStart = QPushButton("요청 시작", self)
        btnStart.move(20, 20)
        btnStart.clicked.connect(self.btnStart_clicked)
 
        btnStop = QPushButton("요청 종료", self)
        btnStop.move(20, 70)
        btnStop.clicked.connect(self.btnStop_clicked)
 
        btnExit = QPushButton("종료", self)
        btnExit.move(20, 120)
        btnExit.clicked.connect(self.btnExit_clicked)
 
    def StopSubscribe(self):
        if self.isSB:
            self.objStockCur1.Unsubscribe()
            self.objStockCur2.Unsubscribe()
            self.objStockCur3.Unsubscribe()
 
        self.isSB = False
 
    def btnStart_clicked(self):
        self.objStockCur1.Subscribe("A003540") # 대신증권
        self.objStockCur2.Subscribe("A000660") # 하이닉스
        self.objStockCur3.Subscribe("A005930") # 삼성전자
 
        print("빼기빼기================-")
        print("실시간 현재가 요청 시작")
        self.isSB = True
 
    def btnStop_clicked(self):
        self.StopSubscribe()
 
 
    def btnExit_clicked(self):
        self.StopSubscribe()
        exit()
 
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()


빼기빼기================-
실시간 현재가 요청 시작
실시간(장중 체결) 삼성전자 150724 46700 대비 900 체결량 1 거래량 19415379
실시간(장중 체결) 삼성전자 150724 46750 대비 950 체결량 42 거래량 19415421
실시간(장중 체결) 삼성전자 150725 46750 대비 950 체결량 4 거래량 19415425
실시간(장중 체결) SK하이닉스 150725 79700 대비 1300 체결량 3 거래량 4943269
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 100 거래량 19415525
실시간(장중 체결) SK하이닉스 150725 79700 대비 1300 체결량 1 거래량 4943270
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 5 거래량 19415530
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 7 거래량 19415537
실시간(장중 체결) SK하이닉스 150725 79600 대비 1200 체결량 1 거래량 4943271
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 50 거래량 19415587
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 1 거래량 19415588
실시간(장중 체결) 삼성전자 150725 46750 대비 950 체결량 174 거래량 19415762
실시간(장중 체결) 삼성전자 150725 46700 대비 900 체결량 2 거래량 19415764
실시간(장중 체결) SK하이닉스 150725 79600 대비 1200 체결량 5 거래량 4943276
실시간(장중 체결) 삼성전자 150725 46750 대비 950 체결량 1 거래량 19415765
실시간(장중 체결) SK하이닉스 150725 79600 대비 1200 체결량 2 거래량 4943278
실시간(장중 체결) 삼성전자 150726 46700 대비 900 체결량 5 거래량 19415770
실시간(장중 체결) 삼성

실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 182 거래량 19425921
실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 182 거래량 19425921
실시간(장중 체결) 삼성전자 150740 46700 대비 900 체결량 2 거래량 19425923
실시간(장중 체결) 삼성전자 150740 46700 대비 900 체결량 2 거래량 19425923
실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 2 거래량 19425925
실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 2 거래량 19425925
실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 29 거래량 19425954
실시간(장중 체결) 삼성전자 150740 46750 대비 950 체결량 29 거래량 19425954
실시간(장중 체결) 삼성전자 150741 46750 대비 950 체결량 5 거래량 19425959
실시간(장중 체결) 삼성전자 150741 46750 대비 950 체결량 5 거래량 19425959
실시간(장중 체결) 삼성전자 150741 46750 대비 950 체결량 300 거래량 19426259
실시간(장중 체결) 삼성전자 150741 46750 대비 950 체결량 300 거래량 19426259
실시간(장중 체결) 삼성전자 150741 46700 대비 900 체결량 300 거래량 19426559
실시간(장중 체결) 삼성전자 150741 46700 대비 900 체결량 300 거래량 19426559
실시간(장중 체결) SK하이닉스 150741 79600 대비 1200 체결량 1 거래량 4944698
실시간(장중 체결) SK하이닉스 150741 79600 대비 1200 체결량 1 거래량 4944698
실시간(장중 체결) SK하이닉스 150741 79600 대비 1200 체결량 2 거래량 4944700
실시간(장중 체결) SK하이닉스 150741 79600 대비 1200 체결량 2 

실시간(장중 체결) 삼성전자 150752 46700 대비 900 체결량 20 거래량 19428527
실시간(장중 체결) 삼성전자 150752 46700 대비 900 체결량 20 거래량 19428527
실시간(장중 체결) SK하이닉스 150752 79600 대비 1200 체결량 52 거래량 4945060
실시간(장중 체결) SK하이닉스 150752 79600 대비 1200 체결량 52 거래량 4945060
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 3 거래량 19428530
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 3 거래량 19428530
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 61 거래량 19428591
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 61 거래량 19428591
실시간(장중 체결) SK하이닉스 150753 79600 대비 1200 체결량 37 거래량 4945097
실시간(장중 체결) SK하이닉스 150753 79600 대비 1200 체결량 37 거래량 4945097
실시간(장중 체결) 삼성전자 150753 46750 대비 950 체결량 30 거래량 19428621
실시간(장중 체결) 삼성전자 150753 46750 대비 950 체결량 30 거래량 19428621
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 10 거래량 19428631
실시간(장중 체결) 삼성전자 150753 46700 대비 900 체결량 10 거래량 19428631
실시간(장중 체결) 삼성전자 150753 46750 대비 950 체결량 213 거래량 19428844
실시간(장중 체결) 삼성전자 150753 46750 대비 950 체결량 213 거래량 19428844
실시간(장중 체결) 삼성전자 150754 46700 대비 900 체결량 4 거래량 19428848
실시간(장중 체결) 삼성전자 150754 46700 대비 900 체결량 4

실시간(장중 체결) SK하이닉스 150801 79600 대비 1200 체결량 125 거래량 4945976
실시간(장중 체결) SK하이닉스 150801 79600 대비 1200 체결량 125 거래량 4945976
실시간(장중 체결) SK하이닉스 150801 79600 대비 1200 체결량 1 거래량 4945977
실시간(장중 체결) SK하이닉스 150801 79600 대비 1200 체결량 1 거래량 4945977
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 50 거래량 19443809
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 50 거래량 19443809
실시간(장중 체결) 대신증권 150802 8690 대비 200 체결량 17 거래량 166425
실시간(장중 체결) 대신증권 150802 8690 대비 200 체결량 17 거래량 166425
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 50 거래량 19443859
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 50 거래량 19443859
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 1 거래량 19443860
실시간(장중 체결) 삼성전자 150802 46750 대비 950 체결량 1 거래량 19443860
실시간(장중 체결) 대신증권 150802 8690 대비 200 체결량 124 거래량 166549
실시간(장중 체결) 대신증권 150802 8690 대비 200 체결량 124 거래량 166549
실시간(장중 체결) 삼성전자 150802 46800 대비 1000 체결량 50 거래량 19443910
실시간(장중 체결) 삼성전자 150802 46800 대비 1000 체결량 50 거래량 19443910
실시간(장중 체결) 삼성전자 150802 46800 대비 1000 체결량 1 거래량 19443911
실시간(장중 체결) 삼성전자 150802 46800 대비 1000 체결량 1 거래량 194

실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 30 거래량 19445824
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 30 거래량 19445824
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 100 거래량 19445924
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 100 거래량 19445924
실시간(장중 체결) SK하이닉스 150811 79600 대비 1200 체결량 10 거래량 4946466
실시간(장중 체결) SK하이닉스 150811 79600 대비 1200 체결량 10 거래량 4946466
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 248 거래량 19446172
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 248 거래량 19446172
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 2 거래량 19446174
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 2 거래량 19446174
실시간(장중 체결) 삼성전자 150811 46750 대비 950 체결량 1 거래량 19446175
실시간(장중 체결) 삼성전자 150811 46750 대비 950 체결량 1 거래량 19446175
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 1 거래량 19446176
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 1 거래량 19446176
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 616 거래량 19446792
실시간(장중 체결) 삼성전자 150811 46800 대비 1000 체결량 616 거래량 19446792
실시간(장중 체결) 삼성전자 150811 46750 대비 950 체결량 3 거래량 19446795
실시간(장중 체결) 삼성전자 150811 46750 대비 9

실시간(장중 체결) SK하이닉스 150819 79600 대비 1200 체결량 10 거래량 4948680
실시간(장중 체결) SK하이닉스 150819 79600 대비 1200 체결량 10 거래량 4948680
실시간(장중 체결) 삼성전자 150819 46800 대비 1000 체결량 1 거래량 19452236
실시간(장중 체결) 삼성전자 150819 46800 대비 1000 체결량 1 거래량 19452236
실시간(장중 체결) 삼성전자 150819 46750 대비 950 체결량 5 거래량 19452241
실시간(장중 체결) 삼성전자 150819 46750 대비 950 체결량 5 거래량 19452241
실시간(장중 체결) 삼성전자 150819 46750 대비 950 체결량 7 거래량 19452248
실시간(장중 체결) 삼성전자 150819 46750 대비 950 체결량 7 거래량 19452248
실시간(장중 체결) 삼성전자 150820 46750 대비 950 체결량 1 거래량 19452249
실시간(장중 체결) 삼성전자 150820 46750 대비 950 체결량 1 거래량 19452249
실시간(장중 체결) 삼성전자 150820 46800 대비 1000 체결량 30 거래량 19452279
실시간(장중 체결) 삼성전자 150820 46800 대비 1000 체결량 30 거래량 19452279
실시간(장중 체결) 삼성전자 150820 46800 대비 1000 체결량 500 거래량 19452779
실시간(장중 체결) 삼성전자 150820 46800 대비 1000 체결량 500 거래량 19452779
실시간(장중 체결) 삼성전자 150820 46750 대비 950 체결량 2 거래량 19452781
실시간(장중 체결) 삼성전자 150820 46750 대비 950 체결량 2 거래량 19452781
실시간(장중 체결) SK하이닉스 150820 79700 대비 1300 체결량 1 거래량 4948681
실시간(장중 체결) SK하이닉스 150820 79700 대비 1300 체결량 1 

실시간(장중 체결) SK하이닉스 150825 79600 대비 1200 체결량 5 거래량 4949115
실시간(장중 체결) SK하이닉스 150825 79600 대비 1200 체결량 5 거래량 4949115
실시간(장중 체결) SK하이닉스 150825 79600 대비 1200 체결량 19 거래량 4949134
실시간(장중 체결) SK하이닉스 150825 79600 대비 1200 체결량 19 거래량 4949134
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 103 거래량 19504437
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 103 거래량 19504437
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 106 거래량 19504543
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 106 거래량 19504543
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 20 거래량 19504563
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 20 거래량 19504563
실시간(장중 체결) 삼성전자 150826 46850 대비 1050 체결량 14 거래량 19504577
실시간(장중 체결) 삼성전자 150826 46850 대비 1050 체결량 14 거래량 19504577
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 20 거래량 19504597
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 20 거래량 19504597
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 10 거래량 19504607
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 10 거래량 19504607
실시간(장중 체결) 삼성전자 150826 46800 대비 1000 체결량 5 거래량 19504612
실시간(장중 체결) 삼성전자 150826 468

실시간(장중 체결) 삼성전자 150829 46800 대비 1000 체결량 13 거래량 19505322
실시간(장중 체결) 삼성전자 150829 46800 대비 1000 체결량 13 거래량 19505322
실시간(장중 체결) 삼성전자 150829 46800 대비 1000 체결량 2 거래량 19505324
실시간(장중 체결) 삼성전자 150829 46800 대비 1000 체결량 2 거래량 19505324
실시간(장중 체결) SK하이닉스 150829 79600 대비 1200 체결량 75 거래량 4963163
실시간(장중 체결) SK하이닉스 150829 79600 대비 1200 체결량 75 거래량 4963163
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 70 거래량 19505394
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 70 거래량 19505394
실시간(장중 체결) 삼성전자 150830 46850 대비 1050 체결량 5 거래량 19505399
실시간(장중 체결) 삼성전자 150830 46850 대비 1050 체결량 5 거래량 19505399
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 1900 거래량 19507299
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 1900 거래량 19507299
실시간(장중 체결) SK하이닉스 150830 79500 대비 1100 체결량 18 거래량 4963181
실시간(장중 체결) SK하이닉스 150830 79500 대비 1100 체결량 18 거래량 4963181
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 5 거래량 19507304
실시간(장중 체결) 삼성전자 150830 46800 대비 1000 체결량 5 거래량 19507304
실시간(장중 체결) 삼성전자 150830 46850 대비 1050 체결량 1 거래량 19507305
실시간(장중 체결) 삼성전자 150830 46850 대